In [1]:
from absbox.local.generic import Generic

test01 = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":1000
             ,"rate":0.07
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"amortizing":[
         ["payFee",["acc01"],['trusteeFee']]
         ,["payInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payResidual","acc01","B"]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None)

In [2]:
from absbox.local.util import mkDealsBy

In [17]:
issuance_plan = [ (1100,0.07),(1500,0.08) ]
total_issuance_bal = 2000

In [20]:
bond_plan = [ {"bonds":(("A1",{"balance":senior_bal
                             ,"rate":senior_r
                             ,"originBalance":senior_bal
                             ,"originRate":0.07
                             ,"startDate":"2020-01-03"
                             ,"rateType":{"Fixed":0.08}
                             ,"bondType":{"Sequential":None}})
                      ,("B",{"balance":(total_issuance_bal - senior_bal)
                             ,"rate":0.0
                             ,"originBalance":(total_issuance_bal - senior_bal)
                             ,"originRate":0.07
                             ,"startDate":"2020-01-03"
                             ,"rateType":{"Fixed":0.00}
                             ,"bondType":{"Equity":None}
                             }))}
        for senior_bal,senior_r in issuance_plan ]



In [25]:
bond_plan_with_name = dict(zip(["SmallSenior","HighSenior"],bond_plan))

differentDeals = mkDealsBy(test01,bond_plan_with_name)

differentDeals['HighSenior']

Generic(name='TEST01', dates={'cutoff': '2021-03-01', 'closing': '2021-06-15', 'firstPay': '2021-07-26', 'payFreq': ['DayOfMonth', 20], 'poolFreq': 'MonthEnd', 'stated': '2030-01-01'}, pool={'assets': [['Mortgage', {'originBalance': 2200, 'originRate': ['fix', 0.045], 'originTerm': 30, 'freq': 'Monthly', 'type': 'Level', 'originDate': '2021-02-01'}, {'currentBalance': 2200, 'currentRate': 0.08, 'remainTerm': 20, 'status': 'current'}]]}, accounts=(('acc01', {'balance': 0}),), bonds=(('A1', {'balance': 1500, 'rate': 0.08, 'originBalance': 1500, 'originRate': 0.07, 'startDate': '2020-01-03', 'rateType': {'Fixed': 0.08}, 'bondType': {'Sequential': None}}), ('B', {'balance': 500, 'rate': 0.0, 'originBalance': 500, 'originRate': 0.07, 'startDate': '2020-01-03', 'rateType': {'Fixed': 0.0}, 'bondType': {'Equity': None}})), fees=(('trusteeFee', {'type': {'fixFee': 30}}),), waterfall={'amortizing': [['payFee', ['acc01'], ['trusteeFee']], ['payInt', 'acc01', ['A1']], ['payPrin', 'acc01', ['A1']],

In [27]:
from absbox import API
localAPI = API("https://absbox.org/api/latest")

import datetime

In [29]:
r = localAPI.runStructs(differentDeals
                      ,read=True
                      ,pricing= {"PVDate":"2021-08-22"
                                ,"PVCurve":[["2021-01-01",0.025]
                                          ,["2024-08-01",0.025]]}
                      ,assumptions=[{"CPR":[0.02,0.02,0.03]}
                                  ,{"CDR":[0.01,0.015,0.021]}
                                  ,{"Recovery":(0.7,18)}]
                      )

In [32]:
r['SmallSenior']['pricing']

,pricing,face,WAL,duration,convexity,accure interest
A1,684.87,62.260909,0.29,0.24,0.37,0
B,1101.42,122.380000,0.87,0.89,1.83,0


In [39]:
r['HighSenior']['pricing']

,pricing,face,WAL,duration,convexity,accure interest
A1,1110.72,74.048,0.45,0.39,0.67,0
B,675.57,135.114,1.04,1.04,2.27,0


In [38]:
r['HighSenior']['bonds']['B']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-07-26,500.00,0.00,0.00,0,0.00,"[<PayPrin:[""B""],Just 500.00>, <PayYield:B,Noth..."
2021-08-20,500.00,0.00,0.00,0,0.00,"[<PayPrin:[""B""],Just 500.00>, <PayYield:B,Noth..."
2021-09-20,500.00,0.00,0.00,0,0.00,"[<PayPrin:[""B""],Just 500.00>, <PayYield:B,Noth..."
2021-10-20,500.00,0.00,0.00,0,0.00,"[<PayPrin:[""B""],Just 500.00>, <PayYield:B,Noth..."
2021-11-20,500.00,0.00,0.00,0,0.00,"[<PayPrin:[""B""],Just 500.00>, <PayYield:B,Noth..."
2021-12-20,500.00,0.00,0.00,0,0.00,"[<PayPrin:[""B""],Just 500.00>, <PayYield:B,Noth..."
2022-01-20,500.00,0.00,0.00,0,0.00,"[<PayPrin:[""B""],Just 500.00>, <PayYield:B,Noth..."
2022-02-20,500.00,0.00,0.00,0,0.00,"[<PayPrin:[""B""],Just 500.00>, <PayYield:B,Noth..."
2022-03-20,500.00,0.00,0.00,0,0.00,"[<PayPrin:[""B""],Just 500.00>, <PayYield:B,Noth..."


In [37]:
r['HighSenior']['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-07-26,1190.07,13.47,309.93,0.08,323.40,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-08-20,1083.33,6.52,106.74,0.08,113.26,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-09-20,977.43,7.36,105.90,0.08,113.26,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-10-20,870.59,6.42,106.84,0.08,113.26,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-11-20,763.24,5.91,107.35,0.08,113.26,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2021-12-20,654.99,5.01,108.25,0.08,113.26,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2022-01-20,546.18,4.45,108.81,0.08,113.26,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2022-02-20,436.63,3.71,109.55,0.08,113.26,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
2022-03-20,326.05,2.67,110.58,0.08,113.25,"[<PayInt:[""A1""],Just 0.00>, <PayPrin:[""A1""],Ju..."
